In [1]:
!pip install -q transformers accelerate peft bitsandbytes datasets trl 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 25.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 92.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 34.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
import os
import json
import random
from pathlib import Path

import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)
import bitsandbytes as bnb

2025-11-16 06:27:04.798230: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763274425.288068      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763274425.421513      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
# Đường dẫn dataset trên Kaggle (bạn có thể thay bằng path thực tế)
PROBLEMS_DIR = "/kaggle/input/pleasework/dataset/problems"
SOLUTIONS_DIR = "/kaggle/input/pleasework/dataset/solution"

def load_samples():
    data = []
    problem_files = sorted(Path(PROBLEMS_DIR).glob("Problem*.md"))
    for pf in problem_files:
        qid = pf.stem.replace("Problem", "")
        sf = Path(SOLUTIONS_DIR) / f"Solution{qid}.py"
        if not sf.exists():
            continue
        problem_text = pf.read_text().strip()
        solution_code = sf.read_text().strip()

        # Format prompt theo kiểu instruct
        prompt = f"""
            <|system|>
            You are an expert Python programmer. You will be given a question (problem specification) and will generate a correct Python program that matches the specification and passes all tests.
            <|user|>
            ### Problem
            {problem_text}
            
            ### Write the solution in Python.
            
            <|assistant|>
            ```python
            {solution_code}
            ```
            """

        data.append({"text": prompt})
    return data

raw_data = load_samples()
print(f"Loaded {len(raw_data)} samples.")
dataset = Dataset.from_list(raw_data)

Loaded 2869 samples.


In [4]:
model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load model in 4-bit
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    trust_remote_code=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Chuẩn bị model cho QLoRA
model = prepare_model_for_kbit_training(model)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [5]:
lora_config = LoraConfig(
    r=64,
    lora_alpha=128,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]  # DeepSeek-Coder dùng kiến trúc LLaMA-like
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 25,165,824 || all params: 1,371,637,760 || trainable%: 1.8347


In [6]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding=False,
        max_length=1024
    )

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset.column_names,
    num_proc=2
)

Map (num_proc=2):   0%|          | 0/2869 [00:00<?, ? examples/s]

In [8]:
training_args = TrainingArguments(
    output_dir="./deepseek-coder-finetuned3b",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    save_strategy="steps",
    save_steps=100,
    logging_steps=10,
    num_train_epochs=3,
    fp16=True,  
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    weight_decay=0.01,
    report_to="none",
    greater_is_better=False,
    save_total_limit=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    dataloader_num_workers=2,
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [9]:
train_dataset = tokenized_dataset

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

/tmp/ipykernel_48/2944723658.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 32021}.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,0.888300
20,0.691300
30,0.531600
40,0.419800
50,0.411300
60,0.399800
70,0.415900
80,0.371500
90,0.365400
100,0.406500


TrainOutput(global_step=540, training_loss=0.3369554016325209, metrics={'train_runtime': 7255.3516, 'train_samples_per_second': 1.186, 'train_steps_per_second': 0.074, 'total_flos': 6.786690676697088e+16, 'train_loss': 0.3369554016325209, 'epoch': 3.0})

In [11]:
trainer.save_model()

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"
adapter_path = "/kaggle/working/deepseek-coder-finetuned3b"  # thư mục bạn lưu bằng trainer.save_model()

# Cấu hình 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# Gắn adapter QLoRA
model = PeftModel.from_pretrained(base_model, adapter_path)
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32256, 2048)
        (layers): ModuleList(
          (0-23): 24 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.

In [17]:
problem_text = r'''
Given two sorted arrays nums1 and nums2 of size m and n respectively, return the median of the two sorted arrays.
The overall run time complexity should be O(log (m+n)).
 
Example 1:

Input: nums1 = [1,3], nums2 = [2]
Output: 2.00000
Explanation: merged array = [1,2,3] and median is 2.

Example 2:

Input: nums1 = [1,2], nums2 = [3,4]
Output: 2.50000
Explanation: merged array = [1,2,3,4] and median is (2 + 3) / 2 = 2.5.

 
Constraints:

nums1.length == m
nums2.length == n
0 <= m <= 1000
0 <= n <= 1000
1 <= m + n <= 2000
-106 <= nums1[i], nums2[i] <= 106


Boilerplate code:
```python
def findMedianSortedArrays(nums1, nums2):
    ...
```
  '''


In [18]:
def generate_solution(problem_text: str, max_new_tokens=300) -> str:
    prompt = f"""
            <|system|>
            You are an expert Python programmer. You will be given a question (problem specification) and will generate a correct Python program that matches the specification and passes all tests.            
            <|user|>
            ### Problem
            {problem_text}
            
            ### Write the solution in Python.
            
            <|assistant|>
            ```python
            """
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens= 512,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    code = generated[len(prompt):]
    if "```" in code:
        code = code.split("```")[0]
    return code.strip()

In [19]:
solution = generate_solution(problem_text)
print(solution)

from typing import List

class Solution:
    def findMedianSortedArrays(self, nums1: List[int], nums2: List[int]) -> float:
        if len(nums1) > len(nums2):
            nums1, nums2 = nums2, nums1
        
        x, y = len(nums1), len(nums2)
        low, high = 0, x
        
        while low <= high:
            partitionX = (low + high) // 2
            partitionY = (x + y + 1) // 2 - partitionX
            
            maxLeftX = float('-inf') if partitionX == 0 else nums1[partitionX - 1]
            minRightX = float('inf') if partitionX == x else nums1[partitionX]
            
            maxLeftY = float('-inf') if partitionY == 0 else nums2[partitionY - 1]
            minRightY = float('inf') if partitionY == y else nums2[partitionY]
            
            if maxLeftX <= minRightY and maxLeftY <= minRightX:
                if (x + y) % 2 == 0:
                    return (max(maxLeftX, maxLeftY) + min(minRightX, minRightY)) / 2
                else:
                    ret

In [12]:
!zip -r /kaggle/working/deepseek-coder-finetuned.zip /kaggle/working/deepseek-coder-finetuned3b


  adding: kaggle/working/deepseek-coder-finetuned3b/ (stored 0%)
  adding: kaggle/working/deepseek-coder-finetuned3b/adapter_model.safetensors (deflated 8%)
  adding: kaggle/working/deepseek-coder-finetuned3b/training_args.bin (deflated 51%)
  adding: kaggle/working/deepseek-coder-finetuned3b/checkpoint-540/ (stored 0%)
  adding: kaggle/working/deepseek-coder-finetuned3b/checkpoint-540/adapter_model.safetensors (deflated 8%)
  adding: kaggle/working/deepseek-coder-finetuned3b/checkpoint-540/training_args.bin (deflated 51%)
  adding: kaggle/working/deepseek-coder-finetuned3b/checkpoint-540/optimizer.pt (deflated 9%)
  adding: kaggle/working/deepseek-coder-finetuned3b/checkpoint-540/tokenizer_config.json (deflated 88%)
  adding: kaggle/working/deepseek-coder-finetuned3b/checkpoint-540/README.md (deflated 66%)
  adding: kaggle/working/deepseek-coder-finetuned3b/checkpoint-540/adapter_config.json (deflated 56%)
  adding: kaggle/working/deepseek-coder-finetuned3b/checkpoint-540/chat_templat